# Etude de la gravitationnale collapse


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Constantes
G = 6.67430e-11  # m³/(kg·s²)
c = 2.99792458e8  # m/s
M_sun = 1.989e30  # kg
R_sun = 6.96e8    # m

## Etude statique et systeme tos

## Etude de la dynamique de la surface--chute libre

Etudions dans un premier temps dans la coordonée temporelle de Schwarzchild la dynamique de la surface. A la surface, comme la masse contenue par dessous est constante et que l'on suppose le vide à l'exterieur, on a simplement a étudier une chute libre. On suit encore une metrique de schwarzchild par le theoreme de OIEggfhoz

On commence par se doter du lagrangien correspondant : $\mathcal{L}=\sqrt{-g_{\mu\nu}u^{\mu}u^{\nu}}$ que l'on parametrise par $\tau$ pour 